In [105]:
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

Promedio:  2.0283186130195707
Esperanza al largo plazo:  2.0


In [137]:
# Parámetros
np.random.seed(6)
n = 10000
phi = 0.5
c = 1
sigmaSqr = 1
mean = c / (1 - phi)
std = sigmaSqr / (1 - phi**2)
eps = np.random.normal(0, sigmaSqr, n)

# Generación de datos.
Y = np.empty(n)
Y[0] = c + eps[0]
for i in range(1, n):
    Y[i] = c + phi * Y[i - 1] + eps[i]

# Visualización de los datos.
plt.title("Muestra de tamaño "  + str(n))
plt.xlabel('Tiempo')
plt.ylabel('Estado')
plt.plot(Y, '*', label='Datos')
plt.plot(np.ones(n) * mean, '--k', label='Esperanza largo plazo')
plt.legend(loc='lower center')
plt.grid()
plt.show()
print("Promedio: ", np.mean(Y))
print('Esperanza al largo plazo: ', mean)

In [138]:
# Maximizar la log-verosimilitud es equivalente a minimizarla con signo menos.
def minusLogVer(theta):
    c = theta[0]
    phi = theta[1]
    sigmaSqr = theta[2]
    return (n / 2) * np.log(2 * np.pi * sigmaSqr) + (1 / (2 * sigmaSqr)) * sum((Y[i] - c - phi*Y[i - 1])**2 
                                                                               for i in range(1, n)) 
# Optimizamos.
bound = ((None, None), (None, None), (1**(-10000), None))
opti = minimize(minusLogVer, [0, 10, 1], method='SLSQP', bounds=bound)
df = pd.DataFrame()
df['Nombres'] = ['C', 'phi', 'sigmaSqr']
df['Parám. Reales'] = [c, phi, sigmaSqr] 
df['Max. velosimilitud'] = [opti.x[0], opti.x[1], opti.x[2]]
print(df)

    Nombres  Parám. Reales  Max. velosimilitud
0         C            1.0            1.002671
1       phi            0.5            0.505726
2  sigmaSqr            1.0            1.000000


In [139]:
# Prior uniforme:
c1 = -3
c2 = 3
phi1 = -1
phi2 = 1
sigmaSqr1 = 1 / 2
sigmaSqr2 = 2

# Función del prior uniforme
def logPrior1(theta):
    # Retorna valor proporcional -log(p(theta)). Con prior uniforme.
    c = theta[0]
    phi = theta[1]
    sigmaSqr = theta[2]
    if c1 <= c <= c2 and phi1 < phi < phi2 and sigmaSqr1 <= sigmaSqr <= sigmaSqr2:
        return -np.log((c2 - c1) * (phi2 - phi1) * (sigmaSqr2 - sigmaSqr1))
    else:
        return 0
# Negative log likelihood.
def NLL1(theta):
    # Negative log likelihood.
    c = theta[0]
    phi = theta[1]
    sigmaSqr = theta[2]
    return minusLogVer(theta) - logPrior1(theta)

# Optimizamos.
bound = ((c1, c2), (phi1, phi2), (sigmaSqr1, sigmaSqr2))
opti = minimize(NLL1, [1.4, 1.5, 1.8], method='SLSQP', bounds=bound)
df = pd.DataFrame()
df['Nombres'] = ['C', 'phi', 'sigmaSqr']
df['Parám. Reales'] = [c, phi, sigmaSqr] 
df['MAP'] = [opti.x[0], opti.x[1], opti.x[2]]
print(df)
    

    Nombres  Parám. Reales       MAP
0         C            1.0  1.002669
1       phi            0.5  0.505726
2  sigmaSqr            1.0  0.985241


# Parte f
Asumiremos que tenemos un valor más probable dentro de cada intervalo. Esta nueva información se traducirá en que nuestras nuevas prior serán densidades desplazadas al intervalo correspondiente y parámetros $\alpha$ y $\beta$ de forma que la esperanza de la nueva variable sea justamente el valor al cual se cree más cercano el parámetro.

In [140]:
# Tomamos los valores reales para comparar.
cProb = c
amp0 = (cProb - c1) / (c2 - cProb)
phiProb = phi
amp1 = (phiProb - phi1) / (phi2 - phiProb)
sigmaSqrProv = sigmaSqr
amp2 = (sigmaSqrProv - sigmaSqr1) / (sigmaSqr2 - sigmaSqrProv)
beta = 1

def betaDesplazada(x, alpha, beta, a, b):
    # Retorna densidad de Z = a + (b - a)*X evaluada en x donde X es Beta(alpha, beta)
    delta = b - a
    x = (x - a) / delta
    return stats.beta.pdf(x, alpha, beta) * (1 / delta)

def logPrior2(theta):
    # Retorna valor proporcional -log(p(theta)). Con prior Beta desplazada.
    c = theta[0]
    amp0 = cProb
    phi = theta[1]
    sigmaSqr = theta[2]
    if c1 <= c <= c2 and phi1 < phi < phi2 and sigmaSqr1 <= sigmaSqr <= sigmaSqr2:
        dens = betaDesplazada(c, beta *  amp0, beta, c1, c2) * betaDesplazada(phi, beta *  amp1, beta, phi1, phi2) * betaDesplazada(sigmaSqr, beta *  amp2, beta, sigmaSqr1, sigmaSqr2)
        return -np.log(dens)
    else:
        return 0
    
# Negative log likelihood.
def NLL2(theta):
    # Negative log likelihood.
    c = theta[0]
    phi = theta[1]
    sigmaSqr = theta[2]
    return minusLogVer(theta) - logPrior(theta)

# Optimizamos.
bound = ((c1, c2), (phi1, phi2), (sigmaSqr1, sigmaSqr2))
opti = minimize(NLL2, [1.4, 1.5, 1.8], method='SLSQP', bounds=bound)
df = pd.DataFrame()
df['Nombres'] = ['C', 'phi', 'sigmaSqr']
df['Parám. Reales'] = [c, phi, sigmaSqr] 
df['MAP'] = [opti.x[0], opti.x[1], opti.x[2]]
print(df)

    Nombres  Parám. Reales       MAP
0         C            1.0  1.002868
1       phi            0.5  0.505627
2  sigmaSqr            1.0  0.985440
